In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/Pushpendra/PCG_Heart/PCG_Dataset

Aortic_Stenosis  Mitral_Regurgitation  Mitral_Stenosis	Mitral_Valve_Prolapse  Normal


In [ ]:
# !pip install pyemd python_speech_features

In [ ]:
# !pip install EMD-signal

In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import librosa
# from PyEMD import EMD

In [ ]:
!pip show librosa

Name: librosa
Version: 0.10.1
Summary: Python module for audio and music processing
Home-page: https://librosa.org
Author: Brian McFee, librosa development team
Author-email: brian.mcfee@nyu.edu
License: ISC
Location: /usr/local/lib/python3.10/dist-packages
Requires: audioread, decorator, joblib, lazy-loader, msgpack, numba, numpy, pooch, scikit-learn, scipy, soundfile, soxr, typing-extensions
Required-by: 


In [ ]:
# Define the labels
labels = ['Aortic_Stenosis', 'Mitral_Regurgitation', 'Mitral_Stenosis', 'Mitral_Valve_Prolapse', 'Normal']

# Define the path to your dataset
data_dir = '/content/drive/MyDrive/Pushpendra/PCG_Heart/PCG_Dataset'

In [ ]:
# Preprocess data
# def preprocess_data(dir, labels):
#     X = []
#     y = []
#     for label in labels:
#         class_dir = os.path.join(dir, label)
#         for file in os.listdir(class_dir):
#             file_path = os.path.join(class_dir, file)
#             signal, sr = librosa.load(file_path)
#             emd = EMD()
#             imfs = emd.emd(signal)
#             mfccs = librosa.feature.mfcc(y=imfs[0], sr=sr, n_mfcc=13)
#             X.append(mfccs.mean(axis=1))
#             y.append(label)
#     return np.array(X), np.array(y)
def preprocess_data(dir, labels):
    X = []
    y = []
    for label in labels:
        class_dir = os.path.join(dir, label)
        for file in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file)
            signal, sr = librosa.load(file_path)

            # Compute the Short-Time Fourier Transform (STFT)
            stft = np.abs(librosa.stft(signal))

            # Compute spectral centroid and bandwidth
            spectral_centroids = librosa.feature.spectral_centroid(S=stft, sr=sr)
            spectral_bandwidth = librosa.feature.spectral_bandwidth(S=stft, sr=sr)

            # Concatenate the features and take the mean
            features = np.concatenate((spectral_centroids, spectral_bandwidth), axis=0).mean(axis=1)

            X.append(features)
            y.append(label)

    return np.array(X), np.array(y)

In [ ]:
X, y = preprocess_data(data_dir, labels)

In [ ]:
np.savetxt('labels_pcg.txt', y, fmt='%s')

In [ ]:
# Encode labels
le = LabelEncoder()
y = to_categorical(le.fit_transform(y))

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization

# Reshape data for LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define model
# model = Sequential()
# model.add(LSTM(256, input_shape=(X_train.shape[1], 1)))
# model.add(Dense(256, activation='relu'))
# model.add(Dense(len(labels), activation='softmax'))

# # Compile model
# model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

def build_improved_model(input_shape, num_classes):
    model = Sequential()

    # Adding the first LSTM layer with some Dropout regularization
    model.add(LSTM(units=256, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    # Adding a second LSTM layer with some Dropout regularization
    model.add(LSTM(units=128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    # Adding a third LSTM layer with some Dropout regularization
    model.add(LSTM(units=64))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    # Adding a Dense layer with some Dropout regularization
    model.add(Dense(units=64, activation='relu'))
    model.add(Dropout(0.2))

    # Output layer
    model.add(Dense(units=num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
# Train model
# model.fit(X_train, y_train, batch_size=32, epochs=80, validation_data=(X_test, y_test))
# Assuming X_train.shape[1] is the number of timesteps and 'labels' is a list of class labels
input_shape = (X_train.shape[1], 1)
num_classes = len(labels)
improved_model = build_improved_model(input_shape, num_classes)

# Now you can train the improved_model with your training data
improved_model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/200
25/25 [==============================] - 10s 84ms/step - loss: 1.8554 - accuracy: 0.2350 - val_loss: 1.6082 - val_accuracy: 0.1850
Epoch 2/200
25/25 [==============================] - 1s 28ms/step - loss: 1.7062 - accuracy: 0.2775 - val_loss: 1.6061 - val_accuracy: 0.1850
Epoch 3/200
25/25 [==============================] - 1s 28ms/step - loss: 1.6631 - accuracy: 0.2663 - val_loss: 1.6043 - val_accuracy: 0.1850
Epoch 4/200
25/25 [==============================] - 1s 27ms/step - loss: 1.5211 - accuracy: 0.3350 - val_loss: 1.6042 - val_accuracy: 0.2850
Epoch 5/200
25/25 [==============================] - 1s 27ms/step - loss: 1.5428 - accuracy: 0.3212 - val_loss: 1.6030 - val_accuracy: 0.2200
Epoch 6/200
25/25 [==============================] - 1s 27ms/step - loss: 1.5014 - accuracy: 0.3313 - val_loss: 1.6052 - val_accuracy: 0.3000
Epoch 7/200
25/25 [==============================] - 1s 22ms/step - loss: 1.5178 - accuracy: 0.3150 - val_loss: 1.5996 - val_accuracy: 0.2800
Epoch

In [ ]:
# Save the model
improved_model.save('PCG_Heart.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
!cp PCG_Heart.h5 /content/drive/MyDrive

In [ ]:
from tensorflow.keras.models import load_model
# Load the saved model
loaded_model = load_model('PCG_Heart.h5')

In [ ]:
# prompt: write code to reload the model and use it for prediction if a .wav file is given as input

import numpy as np
# Load the saved model
# loaded_model = load_model('/content/drive/MyDrive/PCG_Heart.h5')

# Define a function to predict the class of a given audio file
def predict_audio(filename):
    # Load the audio file
    signal, sr = librosa.load(filename)

    # Preprocess the audio file
    stft = np.abs(librosa.stft(signal))
    spectral_centroids = librosa.feature.spectral_centroid(S=stft, sr=sr)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(S=stft, sr=sr)
    features = np.concatenate((spectral_centroids, spectral_bandwidth), axis=0).mean(axis=1)

    # Reshape the features for the model
    features = features.reshape(1, features.shape[0], 1)

    # Make a prediction using the loaded model
    prediction = loaded_model.predict(features)

    # Get the class label corresponding to the prediction
    predicted_class = le.inverse_transform([np.argmax(prediction)])

    # Return the predicted class
    return predicted_class[0]

# Example usage
filename = '/content/drive/MyDrive/Pushpendra/PCG_Heart/PCG_Dataset/Aortic_Stenosis/New_AS_003.wav'
predicted_class = predict_audio(filename)
print(f"Predicted class for {filename}: {predicted_class}")


1/1 [==============================] - 0s 37ms/step
Predicted class for /content/drive/MyDrive/Pushpendra/PCG_Heart/PCG_Dataset/Aortic_Stenosis/New_AS_003.wav: Aortic_Stenosis
